In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np

### 下載資料並進行整理

In [2]:
!wget https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat

--2020-09-03 02:47:11--  https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat [following]
--2020-09-03 02:47:11--  https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152437 (149K) [application/octet-stream]
Saving to: ‘Data.mat’

Data.mat            100%[===================>] 148.86K  --.-KB/s    in 0.02s   

2020-09-03 02:47:12 (6.15 MB/s) - ‘Data.mat’ saved [152437/152437]



In [3]:
data = sio.loadmat('Data.mat')

In [4]:
def to_np_array(data):
  nparr = []

  for _datas in data:
    _arr = []
    for _data in _datas:
      _arr.append(_data)
    nparr.append(_arr)

  return np.array(nparr)

In [5]:
channel1 = to_np_array(data['c1']).transpose((1,0,2,3))
channel2 = to_np_array(data['c2']).transpose((1,0,2,3))
e = 0.1
label = to_np_array(data['lable']).transpose((1,0,2,3))
label = np.maximum(label-e,0)
label = np.maximum(label,e/(label.shape[3]-1))

### 建立模型

In [6]:
def build_model(output):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(output,tf.keras.activations.softmax),
  ])
  model.compile(tf.optimizers.Adam(),tf.losses.categorical_crossentropy,['accuracy'])
  return model

### Model 1
只使用 channel 1

#### 訓練

In [7]:
models_1 = [] 
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel1[:(i)*3],channel1[(i+1)*3:]]).reshape((-1,channel1.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = channel1[i*3:(i+1)*3].reshape((-1,channel1.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_1.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9751 - accuracy: 0.7350
Epoch 5/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9407 - accuracy: 0.7394
Epoch 6/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9225 - accuracy: 0.7511
Epoch 7/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9148 - accuracy: 0.7539
Epoch 8/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9154 - accuracy: 0.7650
Epoch 9/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8934 - accuracy: 0.7817
Epoch 10/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8783 - accuracy: 0.7894
Epoch 11/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8697 - accuracy: 0.7944
Epoch 12/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8585 - accuracy: 0.7950
Epoch 13/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8622 - ac

#### 測試

In [31]:
models_1_accuracy = []
for idx, model in enumerate(models_1):
  x_test = channel1[idx*3:(idx+1)*3].reshape((-1,channel1.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  models_1_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          tf.argmax(model.predict(x_test),axis=1),
          tf.argmax(y_test,axis=1)
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_1_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_1_accuracy))

fold 0 accuracy
tf.Tensor(0.7511111, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.8333333, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.72888887, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.7911111, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.81777775, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.78444445, shape=(), dtype=float32)


### Model 2
只使用 channel 2

#### 訓練

In [8]:
models_2 = []
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel2[:(i)*3],channel2[(i+1)*3:]]).reshape((-1,channel2.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = channel2[i*3:(i+1)*3].reshape((-1,channel2.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_2.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 4ms/step - loss: 1.1221 - accuracy: 0.6478
Epoch 5/500
57/57 [==============================] - 0s 4ms/step - loss: 1.0719 - accuracy: 0.6683
Epoch 6/500
57/57 [==============================] - 0s 4ms/step - loss: 1.0532 - accuracy: 0.6711
Epoch 7/500
57/57 [==============================] - 0s 4ms/step - loss: 1.0394 - accuracy: 0.6744
Epoch 8/500
57/57 [==============================] - 0s 4ms/step - loss: 1.0006 - accuracy: 0.7050
Epoch 9/500
57/57 [==============================] - 0s 4ms/step - loss: 1.0176 - accuracy: 0.6933
Epoch 10/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9873 - accuracy: 0.7117
Epoch 11/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9738 - accuracy: 0.7211
Epoch 12/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9739 - accuracy: 0.7194
Epoch 13/500
57/57 [==============================] - 0s 4ms/step - loss: 0.9492 - ac

#### 測試

In [33]:
models_2_accuracy = []
for idx, model in enumerate(models_2):
  x_test = channel2[idx*3:(idx+1)*3].reshape((-1,channel2.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  models_2_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          tf.argmax(model.predict(x_test),axis=1),
          tf.argmax(y_test,axis=1)
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_2_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_2_accuracy))

fold 0 accuracy
tf.Tensor(0.91333336, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(1.0, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.97333336, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.9266667, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.98, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.9586667, shape=(), dtype=float32)


### Model 3
將 channel 1 與 channel 2 一起使用

#### 訓練

In [9]:
models_3 = [] 
for i in range(5):
  connect_data = np.concatenate([channel1,channel2],axis=3)
  model = build_model(5)
  x_train = np.concatenate([connect_data[:(i)*3],connect_data[(i+1)*3:]]).reshape((-1,connect_data.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = connect_data[i*3:(i+1)*3].reshape((-1,connect_data.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_3.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7789 - accuracy: 0.8772
Epoch 5/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7529 - accuracy: 0.8683
Epoch 6/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7392 - accuracy: 0.8861
Epoch 7/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7267 - accuracy: 0.8856
Epoch 8/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7170 - accuracy: 0.8844
Epoch 9/500
57/57 [==============================] - 0s 4ms/step - loss: 0.6966 - accuracy: 0.9033
Epoch 10/500
57/57 [==============================] - 0s 4ms/step - loss: 0.7190 - accuracy: 0.8817
Epoch 11/500
57/57 [==============================] - 0s 4ms/step - loss: 0.6935 - accuracy: 0.8978
Epoch 12/500
57/57 [==============================] - 0s 4ms/step - loss: 0.6821 - accuracy: 0.9033
Epoch 13/500
57/57 [==============================] - 0s 4ms/step - loss: 0.6991 - ac

#### 測試

In [34]:
models_3_accuracy = []
for idx, model in enumerate(models_3):
  connect_data = np.concatenate([channel1,channel2],axis=3)
  x_test = connect_data[idx*3:(idx+1)*3].reshape((-1,connect_data.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  models_3_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          tf.argmax(model.predict(x_test),axis=1),
          tf.argmax(y_test,axis=1)
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_3_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_3_accuracy))

fold 0 accuracy
tf.Tensor(0.92444444, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.9622222, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.9777778, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.92, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.96, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.9488889, shape=(), dtype=float32)


### 總結
依據實驗結果來看，在 channel 1 中有較多不乾淨的資料會影響決策

##### Accuracy
|        |fold 0|fold 1|fold 2|fold 3|fold 4|average|
|----------------|-------|-------|-------|-------|-------|-------|
|channel 1   |0.75111|0.83333|0.72888|0.79111|0.81777|0.78444|
|channel 2   |0.91333|1.00000|0.97333|0.92666|0.98000|0.95866|
|connect channel|0.92444|0.96222|0.97777|0.92000|0.96000|0.94888|

In [40]:
tf.reduce_sum(
  tf.cast(
    tf.equal(
      tf.argmax(channel1.reshape((-1,channel1.shape[3])),1),
      tf.argmax(label.reshape((-1,label.shape[3])),1)
    ),
    'float32'
  )
)/(150*15)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7173333>